# 📘 Face Recognition using Dlib and OpenCV
This notebook demonstrates how to perform face recognition using Dlib and OpenCV with pre-trained models. 

In [ ]:
#  Step 1: Install Dependencies
!apt-get install cmake -y
!pip install dlib opencv-python

In [ ]:
#  Step 2: Import Libraries
import cv2
import dlib
import numpy as np
from google.colab.patches import cv2_imshow
from urllib.request import urlopen
import os

In [ ]:
#  Step 3: Download Pretrained Models
os.makedirs("models", exist_ok=True)
!wget -O models/shape_predictor_68_face_landmarks.dat.bz2 http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk models/shape_predictor_68_face_landmarks.dat.bz2
!wget -O models/dlib_face_recognition_resnet_model_v1.dat.bz2 http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
!bzip2 -dk models/dlib_face_recognition_resnet_model_v1.dat.bz2

In [ ]:
#  Step 4: Load Models
detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")
face_rec_model = dlib.face_recognition_model_v1("models/dlib_face_recognition_resnet_model_v1.dat")

In [ ]:
#  Step 5: Define Face Embedding Function
def get_face_embedding(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)
    if len(faces) == 0:
        print("No face found!")
        return None
    shape = shape_predictor(img_rgb, faces[0])
    face_descriptor = face_rec_model.compute_face_descriptor(img_rgb, shape)
    return np.array(face_descriptor)

In [ ]:
#  Step 6: Upload Sample Images
from google.colab import files
print("Upload a known face image (e.g., known.jpg):")
uploaded = files.upload()
known_face_path = next(iter(uploaded))
print("Upload a test face image (e.g., test.jpg):")
uploaded = files.upload()
test_face_path = next(iter(uploaded))

In [ ]:
#  Step 7: Compare Faces
known_embedding = get_face_embedding(known_face_path)
test_embedding = get_face_embedding(test_face_path)
if known_embedding is not None and test_embedding is not None:
    distance = np.linalg.norm(known_embedding - test_embedding)
    print(f"\nDistance between faces: {distance:.4f}")
    if distance < 0.6:
        print(" Faces Match! Same Person")
    else:
        print(" Faces Do Not Match")